In [1]:
import numpy as np
import pandas as pd
import math
from numpy.linalg import *
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize
%matplotlib inline  

In [2]:
def naivelm():
    #input term
    infname=input("Enter the data file name: ")
    sep=input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")
    if sep=='1':
        form=' '
    elif sep=='2':
        form=','
    else:
        return('You could select only 1 or 2')
    
    hd=input("Select the data header format(1 = with header or 2 = no header): ")
    if hd=='1':
        header='infer'
    elif hd=='2':
        header=None
    else:
        return('You could select only 1 or 2')
 
    cl=input("Select column# where your y vlaues exist: ")
    col = int(cl)-1
    
    data=pd.read_csv(infname,sep=form,header=header)

    #calculate term

    y=data.iloc[:,col]
    x=data.drop(col,1)
    n=data.shape[0]
    p=data.shape[1]
    one=pd.DataFrame(np.ones(n))
    x=pd.concat([one,x],axis=1)

    bhat=inv(x.transpose().dot(x)).dot(x.transpose()).dot(y)
    yhat= x.dot(bhat)

    estout=pd.concat([y,yhat],axis=1)

    j=np.ones((n,n))
    sse=y.transpose().dot(y)-bhat.transpose().dot(x.transpose()).dot(y)
    ssto=y.transpose().dot(y) - (1/n)*y.transpose().dot(j).dot(y)

    mse = sse /(n-p)
    rsquared= 1 - ((n-1)/(n-p))*(sse/ssto)
    mseout=pd.DataFrame([mse,rsquared])
    #output and naming term
    """
    outputm=input('Select your output 1:bhat 2:fittedvalue 3:mse&r-squared')
    if(outputm=='1'):
        bhat=pd.DataFrame(bhat)
        b=[]
        for i in bhat.index:
            b.append('Beta' + str(i))
        b[0]='Constant'
        bhat.index=b
        bhat.columns=['Coefficients']
        return(bhat)
    elif(outputm=='2'):
        estout.columns.name = 'ID'
        estout.columns = ['Actual values','Fitted values']
        return(estout)
    elif(outputm=='3'):
        mseout.index=['R-Squared = ','MSE = ']
        mseout.columns=['Model Summary']
        return(mseout)
    else:
        return('You could select only 1,2 or 3')
    """
    bhat=pd.DataFrame(bhat)
    b=[]
    for i in bhat.index:
        b.append('Beta' + str(i))
    b[0]='Constant'
    bhat.index=b
    bhat.columns=['Coefficients']
    estout.columns.name = 'ID'
    estout.columns = ['Actual values','Fitted values']
    mseout.index=['R-Squared = ','MSE = ']
    mseout.columns=['Model Summary']
    output = np.array([bhat,estout,mseout])
    
    fname=input('Select your output file name: ')
    bhat.to_csv(fname, mode='w',sep=' ' ,header=False)
    estout.to_csv(fname, mode='a', header=True)
    mseout.to_csv(fname, mode='a',sep=' ', header=False)
    return(output)
    #print(bhat,'\n', estout, '\n',mseout)


In [3]:
def startlda():
    #input term
    trfname=input("Write your train data file name : ")
    tstfname=input("Write your test data file name : ")
    sep=input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")
    if sep=='1':
        form=' '
    elif sep=='2':
        form=','


    hd=input("Select the data header format(1 = with header or 2 = no header): ")
    if hd=='1':
        header='infer'
    elif hd=='2':
        header=None


    cl=input("Select column# where your y vlaues exist: ")
    col = int(cl)-1

    train = pd.read_csv(trfname,sep=form,header=header)
    test = pd.read_csv(tstfname,sep=form,header=header)
    #calculate term

    Y=train.iloc[:,col]
    X=train.drop(col,1)
    n=train.shape[0]
    p=X.shape[1]

    testY=test.iloc[:,col]
    testX=test.drop(col,1)
    testn=test.shape[0]

    g=list(set(Y))
    k=len(g)

    x=[]
    testx=[]
    for i in g:
        x.append(train[train.iloc[:,col]==i].drop(col,1))
    for i in g:
        testx.append(test[test.iloc[:,col]==i].drop(col,1))

    xbar=[]
    for i in range(k):
        xbar.append(x[i].mean(axis=0))

    pw=[]
    for i in range(k):
        pw.append(x[i].shape[0]/n)
    S=[]
    for i in range(k):
        S.append(x[i].cov())
    p=X.shape[1]
    sigma = pd.DataFrame(np.zeros((p,p)))
    for i in range(k):
        sigma = sigma + (x[i].shape[0]-1)*S[i]
    Sp=sigma*(1/(n-k))

    #pred function
    def dk(vec):
        out=[]
        for i in range(k):
            out.append(xbar[i].dot(inv(Sp)).dot(vec) -0.5 * (xbar[i].dot(inv(Sp)).dot(xbar[i].transpose())) + math.log(pw[i]))
        return(out)
    def pred(vec):
        out=pd.DataFrame(g).loc[pd.DataFrame([g,vec]).iloc[1]==max(pd.DataFrame([g,vec]).iloc[1])].iloc[0][0]
        return(out)

    ## resub output
    yhat = []
    for i in range(n):
        yhat.append(pred(dk(X.iloc[i])))

    predy = pd.DataFrame([range(1,(n+1)),Y,yhat]).transpose()
    predy.columns=['ID','Actual class','Resub pred']
    cont = pd.crosstab(predy.iloc[:,1],predy.iloc[:,2])
    acc = sum(np.diag(cont))/n
    #acc = pd.DataFrame(acc)
    
    ## test output
    testyhat = []
    for i in range(testn):
        testyhat.append(pred(dk(testX.iloc[i])))

    tpredy = pd.DataFrame([range(1,(testn+1)),testY,testyhat]).transpose()
    tpredy.columns=['ID','Actual class','Test pred']
    tcont = pd.crosstab(tpredy.iloc[:,1],tpredy.iloc[:,2])
    tacc = sum(np.diag(tcont))/testn
    #tacc=pd.DataFrame(tacc)

    fname = input('Write your output file name : ')
    #console output
    print(predy.to_string(index=False))
    print(cont)
    print("Overall accuracy = %0.3f" %(acc))

    print(tpredy.to_string(index=False))
    print(tcont)
    print("Overall accuracy = %0.3f" %(tacc))
    
    #file output
    out = open(fname, 'w')
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(predy.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (Resubstitution)",file=out)
        print(cont,file=out)
        print("",file=out)
        print("Model Summary (Resubstitution)",file=out)
        print("Overall accuracy = %0.3f" %(acc),file=out)
        print("",file=out)
        print(tpredy.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (Test)",file=out)
        print(tcont,file=out)
        print("",file=out)
        print("Model Summary (Test)",file=out)
        print("Overall accuracy = %0.3f" %(tacc),file=out)
    out.close()

In [4]:
def startqda():
    #input term
    trfname=input("Write your train data file name : ")
    tstfname=input("Write your test data file name : ")
    sep=input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")
    if sep=='1':
        form=' '
    elif sep=='2':
        form=','


    hd=input("Select the data header format(1 = with header or 2 = no header): ")
    if hd=='1':
        header='infer'
    elif hd=='2':
        header=None


    cl=input("Select column# where your y vlaues exist: ")
    col = int(cl)-1

    train = pd.read_csv(trfname,sep=form,header=header)
    test = pd.read_csv(tstfname,sep=form,header=header)
    #calculate term

    Y=train.iloc[:,col]
    X=train.drop(col,1)
    n=train.shape[0]
    p=X.shape[1]

    testY=test.iloc[:,col]
    testX=test.drop(col,1)
    testn=test.shape[0]

    g=list(set(Y))
    k=len(g)

    x=[]
    testx=[]
    for i in g:
        x.append(train[train.iloc[:,col]==i].drop(col,1))
    for i in g:
        testx.append(test[test.iloc[:,col]==i].drop(col,1))

    xbar=[]
    for i in range(k):
        xbar.append(x[i].mean(axis=0))

    pw=[]
    for i in range(k):
        pw.append(x[i].shape[0]/n)
    S=[]
    for i in range(k):
        S.append(x[i].cov())
    p=X.shape[1]
    sigma = pd.DataFrame(np.zeros((p,p)))
    for i in range(k):
        sigma = sigma + (x[i].shape[0]-1)*S[i]
    Sp=sigma*(1/(n-k))

    #pred function
    def dk(vec):
        out=[]
        for i in range(k):
            out.append(-0.5*(math.log(det(S[i])))-0.5*((vec-xbar[i]).dot(inv(S[i])).dot((vec-xbar[i]).transpose())) +math.log(pw[i]))
            #out.append(xbar[i].dot(inv(Sp)).dot(vec) -0.5 * (xbar[i].dot(inv(Sp)).dot(xbar[i].transpose())) + math.log(pw[i]))
        return(out)
    def pred(vec):
        out=pd.DataFrame(g).loc[pd.DataFrame([g,vec]).iloc[1]==max(pd.DataFrame([g,vec]).iloc[1])].iloc[0][0]
        return(out)

    ## resub output
    yhat = []
    for i in range(n):
        yhat.append(pred(dk(X.iloc[i])))

    predy = pd.DataFrame([range(1,(n+1)),Y,yhat]).transpose()
    predy.columns=['ID','Actual class','Resub pred']
    cont = pd.crosstab(predy.iloc[:,1],predy.iloc[:,2])
    acc = sum(np.diag(cont))/n

    ## test output
    testyhat = []
    for i in range(testn):
        testyhat.append(pred(dk(testX.iloc[i])))

    tpredy = pd.DataFrame([range(1,(testn+1)),testY,testyhat]).transpose()
    tpredy.columns=['ID','Actual class','Test pred']
    tcont = pd.crosstab(tpredy.iloc[:,1],tpredy.iloc[:,2])
    tacc = sum(np.diag(tcont))/testn


    fname = input('Write your output file name : ')
    #console output
    print(predy.to_string(index=False))
    print(cont)
    print("Overall accuracy = %0.3f" %(acc))

    print(tpredy.to_string(index=False))
    print(tcont)
    print("Overall accuracy = %0.3f" %(tacc))
    
    #file output
    out = open(fname, 'w')
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(predy.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (Resubstitution)",file=out)
        print(cont,file=out)
        print("",file=out)
        print("Model Summary (Resubstitution)",file=out)
        print("Overall accuracy = %0.3f" %(acc),file=out)
        print("",file=out)
        print(tpredy.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (Test)",file=out)
        print(tcont,file=out)
        print("",file=out)
        print("Model Summary (Test)",file=out)
        print("Overall accuracy = %0.3f" %(tacc),file=out)
    out.close()

In [5]:
def startrda():
    #input term
    trfname=input("Write your train data file name : ")
    tstfname=input("Write your test data file name : ")
    sep=input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")
    if sep=='1':
        form=' '
    elif sep=='2':
        form=','


    hd=input("Select the data header format(1 = with header or 2 = no header): ")
    if hd=='1':
        sh=1
    elif hd=='2':
        sh=0


    cl=input("Select column# where your y vlaues exist: ")
    col = int(cl)-1

    train = np.genfromtxt(trfname, delimiter=form, skip_header=sh)
    test = np.genfromtxt(tstfname, delimiter=form, skip_header=sh)

    #calculate term
    X = np.delete(train, col, 1) # input variable
    Y = train[:,col] # target variable

    n = len(train) # number of obs
    p = X.shape[1] # number of variables

    testX = np.delete(test, col, 1) # input variable
    testY = test[:,col] # target variable

    g = np.array(list(set(Y))) # unique value of target variable
    k = len(g) # number of unique values

    #calculate mean of each variables by y values
    tmp=[]
    for i in g:
        tmp.append(X[np.where(train[:,col]==i)])
    xby = np.array(tmp)
    xbar = np.array(list(map(lambda x : np.mean(x,axis=0),xby)))

    #prior probability
    pw=[]
    for i in range(k):
        pw.append(xby[i].shape[0]/n)
    pw = np.array(pw)

    #np.cov(xby[0].transpose())
    S0 = np.array(list(map(lambda x : np.cov(x.transpose()) , xby)))

    # ready for calculate covariance matrix
    sigma =np.zeros((p,p))
    for i in range(k):
        sigma = sigma + (xby[i].shape[0]-1)*S0[i]
    Sp=sigma*(1/(n-k))
    
    def dk(vec, S = S0):
        out=[]
        for i in range(k):
            out.append(-0.5*(math.log(det(S[i])))-0.5*((vec-xbar[i]).dot(inv(S[i])).dot((vec-xbar[i]).transpose())) +math.log(pw[i]))
        return(out)
    
    def dk2(vec, S = S0):
        out=[]
        for i in range(k):
            out.append(-0.5*(math.log(det(S[i])))-0.5*np.diag((vec-xbar[i]).dot(inv(S[i])).dot((vec-xbar[i]).transpose())) +math.log(pw[i]))
        return(out)
    def pred(vec):
        out=pd.DataFrame(g).loc[pd.DataFrame([g,vec]).iloc[1]==max(pd.DataFrame([g,vec]).iloc[1])].iloc[0][0]
        return(out)

    tp = np.array(list(map(dk,X)))
    yhat = np.argmax(tp,axis =1)
    cont = pd.crosstab(Y,yhat)
    acc = sum(np.diag(cont))/n

    #ready for grid search
    s2=np.diag(Sp).mean()

    def getacc(Sk ,a,r):
        Sar = a*Sk + (1 - a)*(r*Sp + (1 - r)*s2*np.identity(p))
        tp2 = np.array(dk2(testX,Sar)).reshape(k,len(testX)).transpose()
        yhatg = np.argmax(tp2,axis =1)
        contg = pd.crosstab(testY,yhatg)
        acc2 = sum(np.diag(contg))/len(test)
        return(acc2)

    df = pd.DataFrame()
    gridseed=21
    for i in range(gridseed):
        df=df.append(pd.DataFrame(np.zeros(gridseed)).transpose())
    alpha = gamma = list(map(lambda x : x/(gridseed-1) , list(range(gridseed))))
    df.columns=alpha
    df.index=gamma
    for a ,i in zip(alpha , range(gridseed)):
        for r, j in zip(gamma, range(gridseed)) :
            df.iloc[i,j]= getacc(S0,a,r)
    #get the optimized value of alpha & gamma (alpha = 0.53, beta = 1)
    argm=df.stack().index[np.argmax(df.values)]
    a = argm[0]
    r = argm[1]
    Snew = a*S0 + (1 - a)*(r*Sp + (1 - r)*s2*np.identity(p))

    #resub output
    tp = np.array(list(map(lambda x : dk(x,Snew),X)))
    yhat = np.argmax(tp,axis =1)
    cont = pd.crosstab(Y,yhat)
    acc = sum(np.diag(cont))/n

    #test output
    ttp = np.array(list(map(lambda x : dk(x,Snew),testX)))
    tyhat = np.argmax(ttp,axis =1)
    tcont = pd.crosstab(testY,tyhat)
    tacc = sum(np.diag(tcont))/len(test)


    predy = pd.DataFrame([range(1,(len(train)+1)),Y,yhat]).transpose()
    predy.columns=['ID','Actual class','Resub pred']

    tpredy = pd.DataFrame([range(1,(len(test)+1)),testY,tyhat]).transpose()
    tpredy.columns=['ID','Actual class','Test pred']

    cont.columns = g
    cont.index = g
    cont.columns.name = 'Resub Class'
    cont.index.name = 'Actual Class'

    tcont.columns = g
    tcont.index = g
    tcont.columns.name = 'Prediction Class'
    tcont.index.name = 'Actual Class'

    fname = input('Write your output file name : ')
    #plotting
    x = df.columns
    y = df.index
    X,Y = np.meshgrid(x,y)
    Z = df
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z)
    plt.show()
    
    #console output
    print(predy.to_string(index=False))
    print(cont)
    print("Overall accuracy = %0.3f" %(acc))

    print(tpredy.to_string(index=False))
    print(tcont)
    print("Overall accuracy = %0.3f" %(tacc))
    
    #file output
    out = open(fname, 'w')
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(predy.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (Resubstitution)",file=out)
        print(cont,file=out)
        print("",file=out)
        print("Model Summary (Resubstitution)",file=out)
        print("Overall accuracy = %0.3f" %(acc),file=out)
        print("",file=out)
        print(tpredy.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (Test)",file=out)
        print(tcont,file=out)
        print("",file=out)
        print("Model Summary (Test)",file=out)
        print("Overall accuracy = %0.3f" %(tacc),file=out)
    out.close()

In [6]:
def startlogi():
    #input term
    trfname=input("Write your train data file name : ")
    tstfname=input("Write your test data file name : ")
    sep=input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")
    if sep=='1':
        form=' '
    elif sep=='2':
        form=','


    hd=input("Select the data header format(1 = with header or 2 = no header): ")
    if hd=='1':
        sh=1
    elif hd=='2':
        sh=0


    cl=input("Select column# where your y vlaues exist: ")
    col = int(cl)-1
    coff=input("Select your cutoff value: ")    
    cutoff = float(coff)
    
    train = np.genfromtxt(trfname, delimiter=form, skip_header=sh)
    test = np.genfromtxt(tstfname, delimiter=form, skip_header=sh)

    #calculate term
    X = np.delete(train, col, 1) # input variable
    Y = train[:,col] # target variable

    n = len(train) # number of obs
    p = X.shape[1] # number of variables

    testX = np.delete(test, col, 1) # input variable
    testY = test[:,col] # target variable
    testn = len(test)

    g = np.array(list(set(Y))) # unique value of target variable
    k = len(g) # number of unique values

    tmp = np.zeros((X.shape[0],X.shape[1]+1))
    tmp[:,0]= np.ones(n)
    tmp[:,1:] = X
    Xj= tmp
    beta = np.ones(p+1)/10
    Yb = np.array(list(map(int,Y==g[1])))

    testmp = np.zeros((testX.shape[0],testX.shape[1]+1))
    testmp[:,0]= np.ones(testn)
    testmp[:,1:] = testX
    testXj = testmp
    testYb = np.array(list(map(int,testY==g[1])))

    def loglike(beta):
        a=-Yb.T.dot(Xj).dot(beta)+sum(np.log(1+np.exp(Xj.dot(beta))))
        return a
    opt = minimize(loglike , beta )
    betahat = opt.x

    def gety(x):
        if x>=cutoff:
            return g[1]
        elif x<cutoff:
            return g[0]

    yhat = np.exp(Xj.dot(betahat))/(1+np.exp(Xj.dot(betahat)))
    Yhat = np.array(list(map(gety,yhat)))

    testyhat = np.exp(testXj.dot(betahat))/(1+np.exp(testXj.dot(betahat)))
    testYhat = np.array(list(map(gety,testyhat))) 

    #resub output
    #tp = np.array(list(map(lambda x : dk(x,Snew),X)))
    #yhat = np.argmax(tp,axis =1)
    cont = pd.crosstab(Y,Yhat)
    acc = sum(np.diag(cont))/n
    spe = cont.iloc[0,0]/sum(cont.iloc[0,:])
    sens = cont.iloc[1,1]/sum(cont.iloc[1,:])
    
    #test output
    #ttp = np.array(list(map(lambda x : dk(x,Snew),testX)))
    #tyhat = np.argmax(ttp,axis =1)
    tcont = pd.crosstab(testY,testYhat)
    tacc = sum(np.diag(tcont))/len(test)
    tspe = tcont.iloc[0,0]/sum(tcont.iloc[0,:])
    tsens = tcont.iloc[1,1]/sum(tcont.iloc[1,:])
    

    predy = pd.DataFrame([range(1,(len(train)+1)),Y,Yhat,yhat]).transpose()
    predy.columns=['ID','Actual class','Resub pred','Pred Prob']

    tpredy = pd.DataFrame([range(1,(len(test)+1)),testY,testYhat,testyhat]).transpose()
    tpredy.columns=['ID','Actual class','Test pred','Pred Prob']

    cont.columns = g
    cont.index = g
    cont.columns.name = 'Resub Class'
    cont.index.name = 'Actual Class'

    tcont.columns = g
    tcont.index = g
    tcont.columns.name = 'Prediction Class'
    tcont.index.name = 'Actual Class'

    fname = input('Write your output file name : ')
    
    #console output
    print(predy.to_string(index=False))
    print(cont)
    print("Overall accuracy = %0.3f" %(acc))
    print("Sensitivity = %0.3f" %(sens))
    print("Specificity = %0.3f" %(spe))

    print(tpredy.to_string(index=False))
    print(tcont)
    print("Overall accuracy = %0.3f" %(tacc))
    print("Sensitivity = %0.3f" %(tsens))
    print("Specificity = %0.3f" %(tspe))
    
    #file output
    out = open(fname, 'w')
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(predy.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (Resubstitution)",file=out)
        print(cont,file=out)
        print("",file=out)
        print("Model Summary (Resubstitution)",file=out)
        print("Overall accuracy = %0.3f" %(acc),file=out)
        print("Sensitivity = %0.3f" %(sens),file=out)
        print("Specificity = %0.3f" %(spe),file=out)
    out.close()

In [17]:
def nbayes():
    trfname=input("Write your train data file name : ")
    #tstfname=input("Write your test data file name : ")
    sep=input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")
    if sep=='1':
        form=' '
    elif sep=='2':
        form=','


    hd=input("Select the data header format(1 = with header or 2 = no header): ")
    if hd=='1':
        header='infer'
    elif hd=='2':
        header=None

    data= pd.read_csv(trfname, header = header , sep = form)

    cl=input("Select column# where your y vlaues exist: ")
    yCol = int(cl)-1
    coff=input("Select your cutoff value: ")    
    cutoff = float(coff)
    misVal = input("input shape of your missing data: ")

    Xraw = data.drop(yCol,1)
    Y = data.iloc[:, yCol]

    #test Y have 2 category
    if len(set(Y))!=2:
        return('you must input data with binary y value')
    # let max(Y) = 1 (True) , min(Y) = 0 (False)

    #treat the missing value
    X = Xraw.replace('?',np.nan).apply(pd.to_numeric)
    n = X.shape[0]

    # count the number of unique values, when it is less or equal than 10 treat the variable as categorical
    unqCnt = []
    for i in range(X.shape[1]):
        unqCnt += [len(set(X.iloc[:,i]))]
    uqCnt = np.array(unqCnt)

    cateVarInd = np.where(uqCnt<=10)[0] # index of the categoral values
    numerVarInd = np.where(uqCnt>10)[0] # index of the numerical values

    # for gaussian naive bayes we need to calculate the variance and mean of numerical variables
    X1 = X[list(map(bool, Y - min(Y)))] # Y = max(Y)
    X0 = X[list(map(bool, Y - max(Y)))] # Y = min(Y)
    #True
    mu1 = dict(np.mean(X1,axis = 0)[numerVarInd])
    std1 = dict(np.std(X1,axis = 0)[numerVarInd])
    #False
    mu0 = dict(np.mean(X0,axis = 0)[numerVarInd])
    std0 = dict(np.std(X0,axis = 0)[numerVarInd])


    # when true X ~ N (mu1, var1)
    # P( X =x | True) =  (1/(sqrt(2pie)*std))*exp((-(x-mu)^2)/(2*std^2))

    # make probability table
    cst= dict()
    for i in cateVarInd:
        cst.update({i:pd.crosstab(X.iloc[:,i],Y)})
        for j in range(2):
            cst[i].iloc[:,j] = cst[i].iloc[:,j]/sum(cst[i].iloc[:,j])

    # calculate prob
    def calcDPT(row):
        p = 1
        for j in cateVarInd:
            if np.isnan(row[j]):
                p = p
            else:
                p = p * cst[j].loc[row[j]][max(Y)]
        return(p)

    def calcDPF(row):
        p = 1
        for j in cateVarInd:
            if np.isnan(row[j]):
                p = p
            else:
                p = p * cst[j].loc[row[j]][min(Y)]
        return(p)

    def calcNPT(row):
        p = 1
        for j in numerVarInd:
            if np.isnan(row[j]):
                p = p
            else:
                p = p * (1/(((2*np.pi)**0.5)*std1[j]))*np.exp((-(row[j]-mu1[j])**2)/(2*std1[j]**2))
        return(p)

    def calcNPF(row):
        p = 1
        for j in numerVarInd:
            if np.isnan(row[j]):
                p = p
            else:
                p = p * (1/(((2*np.pi)**0.5)*std0[j]))*np.exp((-(row[j]-mu0[j])**2)/(2*std0[j]**2))
        return(p)

    def calcT(row):
        return(calcDPT(row)*calcNPT(row))

    def calcF(row):
        return(calcDPF(row)*calcNPF(row))

    def getTprob(row):
        return calcT(row)/(calcT(row)+calcF(row))

    predProb = []
    for i in range(X.shape[0]):
        predProb.append(getTprob(X.iloc[i,:]))

    def gety(x):
        if x>=cutoff:
            return max(Y)
        elif x<cutoff:
            return min(Y)

    Yhat = np.array(list(map(gety,predProb)))
    yprob = list(map(lambda x : round(x,3),predProb))

    cont = pd.crosstab(Y,Yhat)
    acc = sum(np.diag(cont))/n
    spe = cont.iloc[0,0]/sum(cont.iloc[0,:])
    sens = cont.iloc[1,1]/sum(cont.iloc[1,:])
    predy = pd.DataFrame([range(1,(len(X)+1)),Y,Yhat,yprob]).transpose()
    predy.columns=['ID','Actual class','Resub pred','Pred Prob']
    cont.columns.name = 'Resub Class'
    cont.index.name = 'Actual Class'
    fname = input('Write your output file name : ')
    #console output
    print(predy.to_string(index=False))
    print(cont)
    print("Overall accuracy = %0.3f" %(acc))
    print("Sensitivity = %0.3f" %(sens))
    print("Specificity = %0.3f" %(spe))

    #file output

    out = open(fname, 'w')
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(predy.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (Resubstitution)",file=out)
        print(cont,file=out)
        print("",file=out)
        print("Model Summary (Resubstitution)",file=out)
        print("Overall accuracy = %0.3f" %(acc),file=out)
        print("Sensitivity = %0.3f" %(sens),file=out)
        print("Specificity = %0.3f" %(spe),file=out)
    out.close()

In [13]:
def startclf():
    inp = input('Select Classification model 1 = LDA , 2 = QDA, 3 = RDA, 4 = logistic regression, 5 = Naive Bayes : ')
    if inp=='1':
        return(startlda())
    elif inp=='2':
        return(startqda())
    elif inp=='3':
        return(startrda())
    elif inp=='4':
        return(startlogi())
    elif inp=='5':
        return(nbayes())

In [14]:
def start():
    inp = input('Select model 1 = Linear regression , 2 = Classification : ')
    if inp=='1':
        return(naivelm())
    if inp=='2':
        return(startclf())

In [18]:
start()

Select model 1 = Linear regression , 2 = Classification : 2
Select Classification model 1 = LDA , 2 = QDA, 3 = RDA, 4 = logistic regression, 5 = Naive Bayes : 5
Write your train data file name : heart.dat
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): 1
Select the data header format(1 = with header or 2 = no header): 2
Select column# where your y vlaues exist: 14
Select your cutoff value: 0.5
input shape of your missing data?
ID  Actual class  Resub pred  Pred Prob
  1.0           1.0         1.0      0.067
  2.0           1.0         1.0      0.001
  3.0           1.0         1.0      0.000
  4.0           1.0         1.0      0.479
  5.0           1.0         2.0      0.957
  6.0           1.0         1.0      0.000
  7.0           1.0         1.0      0.063
  8.0           1.0         1.0      0.031
  9.0           1.0         1.0      0.000
 10.0           1.0         1.0      0.006
 11.0           1.0         1.0      0.036
 12.0           1.0         1.0      0.000
 1

Write your output file name : hw66.txt
